# PyTorch TL DBC

In [1]:
import torch
import torch.nn as nn
import numpy as np
import torch.optim as optim
import torch.nn.functional as F
from tqdm.notebook import tqdm
from torchvision import transforms
from torchvision.datasets import ImageFolder
import torchvision.models as models
from torch.utils.data import DataLoader
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [2]:
batch_size = 20

In [3]:
data_transforms = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

image_datasets = {
    'train': ImageFolder(root='dogImages/train', transform=data_transforms),
    'valid': ImageFolder(root='dogImages/valid', transform=data_transforms),
    'test':  ImageFolder(root='dogImages/test',  transform=data_transforms)
}

data_loaders = {
    'train': DataLoader(image_datasets['train'], batch_size=batch_size, shuffle=True),
    'valid': DataLoader(image_datasets['valid'], batch_size=batch_size),
    'test':  DataLoader(image_datasets['test'],  batch_size=batch_size)
}

print('Train images', len(image_datasets['train']))
print('Valid images', len(image_datasets['valid']))
print('Test images', len(image_datasets['test']))
dog_classes = len(image_datasets['train'].classes)
print('Dog classes', dog_classes)
steps_per_epoch = len(image_datasets['train'])/20
print('Epoch steps', steps_per_epoch)

Train images 6680
Valid images 835
Test images 836
Dog classes 133
Epoch steps 334.0


In [5]:
model = models.mobilenet_v2(pretrained=True)

Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /home/ubuntu/.cache/torch/checkpoints/mobilenet_v2-b0353104.pth
100%|██████████| 13.6M/13.6M [00:02<00:00, 6.46MB/s]


In [8]:
model

MobileNetV2(
  (features): Sequential(
    (0): ConvBNReLU(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=Tr

In [7]:
model.classifier = nn.Sequential(
    nn.Dropout(0.2),
    nn.Linear(in_features=1280, out_features=dog_classes),
)

In [9]:
# Freeze training for all "features" layers
for param in model.features.parameters():
    param.requires_grad = False

In [10]:
use_cuda = torch.cuda.is_available()
if use_cuda:
    model.cuda()
    print('CUDA active')
else:
    print('CUDA inactive')

CUDA active


In [11]:
def train(n_epochs, loaders, model, optimizer, criterion, save_path):
    valid_loss_min = np.Inf
    
    for epoch in range(1, n_epochs+1):
        train_loss = 0.0
        valid_loss = 0.0
        
        model.train()
        for batch_idx, (data, labels) in tqdm(enumerate(loaders['train']), total=steps_per_epoch):
            if use_cuda:
                data, labels = data.cuda(), labels.cuda()
            
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
            train_loss += (1 / (batch_idx + 1)) * (loss.data - train_loss)
            
        model.eval()
        for batch_idx, (data, labels) in enumerate(loaders['valid']):
            if use_cuda:
                data, labels = data.cuda(), labels.cuda()
                
            output = model(data)
            loss = criterion(output, labels)
            valid_loss += (1 / (batch_idx + 1)) * (loss.data - valid_loss)
            
        train_loss /= len(loaders['train'].dataset)
        valid_loss /= len(loaders['valid'].dataset)
        
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
        ))
        
        if valid_loss <= valid_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
                valid_loss_min,
                valid_loss
            ))
            torch.save(model.state_dict(), save_path)
            valid_loss_min = valid_loss
            
    return model

In [12]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adadelta(model.parameters())

In [13]:
epochs = 5

train(epochs, data_loaders, model, optimizer, criterion, 'pytorch_tl_dbc.pt')


Epoch: 1 	Training Loss: 0.000269 	Validation Loss: 0.000922
Validation loss decreased (inf --> 0.000922).  Saving model ...



Epoch: 2 	Training Loss: 0.000098 	Validation Loss: 0.000757
Validation loss decreased (0.000922 --> 0.000757).  Saving model ...



Epoch: 3 	Training Loss: 0.000074 	Validation Loss: 0.000691
Validation loss decreased (0.000757 --> 0.000691).  Saving model ...



Epoch: 4 	Training Loss: 0.000063 	Validation Loss: 0.000676
Validation loss decreased (0.000691 --> 0.000676).  Saving model ...



Epoch: 5 	Training Loss: 0.000054 	Validation Loss: 0.000726


MobileNetV2(
  (features): Sequential(
    (0): ConvBNReLU(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=Tr

In [14]:
def test(loaders, model, criterion):
    test_loss = 0.0
    correct = 0.0
    total = 0.0
    
    model.eval()
    
    for batch_idx, (data, labels) in enumerate(loaders['test']):
        if use_cuda:
            data, labels = data.cuda(), labels.cuda()

        output = model(data)
        loss = criterion(output, labels)
        test_loss += (1 / (batch_idx + 1)) * (loss.data - test_loss)
        pred = output.data.max(1, keepdim=True)[1]
        correct += np.sum(np.squeeze(pred.eq(labels.data.view_as(pred))).cpu().numpy())
        total += data.size(0)
        
    test_loss /= len(loaders['test'].dataset)
            
    print('Test Loss: {:.6f}\n'.format(test_loss))
    
    print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
        100. * correct / total, correct, total))
    

In [15]:
test(data_loaders, model, criterion)

Test Loss: 0.000778


Test Accuracy: 79% (667/836)
